# Task
Implement k-Nearest Neighbors (k-NN), Decision Tree (for Random Forest), Random Forest, and Naive Bayes classifiers from scratch, train and tune them using the provided `X_train`, `y_train`, `X_val`, `y_val` datasets, evaluate their performance on `X_test`, `y_test` using metrics like Accuracy, Precision, Recall, F1-Score, Confusion Matrix, and ROC-AUC, generate relevant visualizations including learning curves, ROC curves, and feature importance plots, and finally, summarize the findings and comparisons of these models.

## Implement k-NN Classifier

### Subtask:
Implement a k-Nearest Neighbors (k-NN) classifier from scratch. This implementation should support both Euclidean and Manhattan distance metrics, incorporate weighted voting based on distance, and allow experimentation with various 'k' values (1, 3, 5, 7, 10).


**Reasoning**:
Define the basic structure of the KNNClassifier class with an `__init__` method to initialize key parameters like `k` and `distance_metric` as specified in the instructions.



In [ ]:
import numpy as np
from collections import Counter

class KNNClassifier:
    def __init__(self, k=5, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric
        self.X_train = None
        self.y_train = None
        print(f"KNNClassifier initialized with k={self.k} and distance_metric='{self.distance_metric}'")

**Reasoning**:
Implement the `_euclidean_distance` and `_manhattan_distance` methods within the `KNNClassifier` class as private helper functions to calculate the distances between two data points, which are required for the k-NN algorithm.



In [ ]:
import numpy as np
from collections import Counter

class KNNClassifier:
    def __init__(self, k=5, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric
        self.X_train = None
        self.y_train = None
        print(f"KNNClassifier initialized with k={self.k} and distance_metric='{self.distance_metric}'")

    def _euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2)**2))

    def _manhattan_distance(self, x1, x2):
        return np.sum(np.abs(x1 - x2))


**Reasoning**:
Implement the `fit` method within the `KNNClassifier` class to store the training data and labels, which is a necessary step before making predictions.



In [ ]:
import numpy as np
from collections import Counter

class KNNClassifier:
    def __init__(self, k=5, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric
        self.X_train = None
        self.y_train = None
        print(f"KNNClassifier initialized with k={self.k} and distance_metric='{self.distance_metric}'")

    def _euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2)**2))

    def _manhattan_distance(self, x1, x2):
        return np.sum(np.abs(x1 - x2))

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        print("KNNClassifier fit with training data.")

**Reasoning**:
Implement the `predict` method within the `KNNClassifier` class, which will handle calculating distances to training points, identifying the k-nearest neighbors, and performing weighted voting to determine the predicted class for each test data point.



In [ ]:
import numpy as np
from collections import Counter

class KNNClassifier:
    def __init__(self, k=5, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric
        self.X_train = None
        self.y_train = None
        print(f"KNNClassifier initialized with k={self.k} and distance_metric='{self.distance_metric}'")

    def _euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2)**2))

    def _manhattan_distance(self, x1, x2):
        return np.sum(np.abs(x1 - x2))

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        print("KNNClassifier fit with training data.")

    def predict(self, X_test):
        predictions = []
        for x_test_point in X_test:
            distances = []
            for i, x_train_point in enumerate(self.X_train):
                if self.distance_metric == 'euclidean':
                    dist = self._euclidean_distance(x_test_point, x_train_point)
                elif self.distance_metric == 'manhattan':
                    dist = self._manhattan_distance(x_test_point, x_train_point)
                else:
                    raise ValueError("Unsupported distance metric. Choose 'euclidean' or 'manhattan'.")
                distances.append((dist, self.y_train.iloc[i]))

            distances.sort(key=lambda x: x[0])
            k_nearest_neighbors = distances[:self.k]

            # Weighted voting
            class_votes = Counter()
            for dist, label in k_nearest_neighbors:
                weight = 1.0 / (dist + 1e-6)  # Add a small constant to avoid division by zero
                class_votes[label] += weight

            if class_votes:
                predicted_class = max(class_votes, key=class_votes.get)
            else:
                # Fallback if no neighbors or weights, e.g., predict the most frequent class in training data
                # This case might happen if k=0 or k_nearest_neighbors is empty (which shouldn't happen with proper k)
                predicted_class = Counter(self.y_train).most_common(1)[0][0]

            predictions.append(predicted_class)
        return np.array(predictions)


## Implement Decision Tree for Random Forest

### Subtask:
Implement a basic Decision Tree algorithm from scratch, which will serve as the building block for the Random Forest. It should support Gini impurity and entropy as split criteria for node decisions.


**Reasoning**:
The first step in implementing the Decision Tree is to define the `TreeNode` class, which will serve as the fundamental structure for each node in the tree, storing essential attributes like feature index, threshold, children, and impurity.



In [ ]:
import numpy as np
from collections import Counter

class TreeNode:
    def __init__(self, feature_index=None, threshold=None, left_child=None, right_child=None, value=None, impurity=None):
        self.feature_index = feature_index  # Index of the feature used for splitting
        self.threshold = threshold          # Threshold value for splitting
        self.left_child = left_child        # Left child node
        self.right_child = right_child      # Right child node
        self.value = value                  # Class value for leaf node (if not None, it's a leaf)
        self.impurity = impurity            # Impurity of the node (e.g., Gini or entropy)


**Reasoning**:
The next step is to define the `DecisionTree` class with its `__init__` method to initialize key parameters such as `max_depth`, `min_samples_split`, and `criterion`, as specified in the instructions for building the decision tree.



In [ ]:
import numpy as np
from collections import Counter

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2, criterion='gini'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion = criterion
        self.root = None # The root node of the decision tree
        print(f"DecisionTree initialized with max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, and criterion='{self.criterion}'")

**Reasoning**:
Implement private helper methods, `_gini` and `_entropy`, within the `DecisionTree` class to calculate Gini impurity and entropy, respectively, for a given set of labels, which are essential for determining the best split criterion.



In [ ]:
import numpy as np
from collections import Counter

class TreeNode:
    def __init__(self, feature_index=None, threshold=None, left_child=None, right_child=None, value=None, impurity=None):
        self.feature_index = feature_index  # Index of the feature used for splitting
        self.threshold = threshold          # Threshold value for splitting
        self.left_child = left_child        # Left child node
        self.right_child = right_child      # Right child node
        self.value = value                  # Class value for leaf node (if not None, it's a leaf)
        self.impurity = impurity            # Impurity of the node (e.g., Gini or entropy)

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2, criterion='gini'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion = criterion
        self.root = None # The root node of the decision tree
        print(f"DecisionTree initialized with max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, and criterion='{self.criterion}'")

    def _gini(self, y):
        m = len(y)
        if m == 0: # Handle empty set of labels
            return 0.0
        counts = Counter(y)
        gini = 1.0
        for label in counts:
            probability = counts[label] / m
            gini -= probability**2
        return gini

    def _entropy(self, y):
        m = len(y)
        if m == 0:
            return 0.0
        counts = Counter(y)
        entropy = 0.0
        for label in counts:
            probability = counts[label] / m
            entropy -= probability * np.log2(probability)
        return entropy


**Reasoning**:
Implement a private helper method, `_find_best_split`, within the `DecisionTree` class. This method will iterate through all features and possible threshold values to determine the optimal split point that yields the lowest impurity (or highest information gain) based on the chosen criterion, which is a fundamental step in building the decision tree.



In [ ]:
import numpy as np
from collections import Counter

class TreeNode:
    def __init__(self, feature_index=None, threshold=None, left_child=None, right_child=None, value=None, impurity=None):
        self.feature_index = feature_index  # Index of the feature used for splitting
        self.threshold = threshold          # Threshold value for splitting
        self.left_child = left_child        # Left child node
        self.right_child = right_child      # Right child node
        self.value = value                  # Class value for leaf node (if not None, it's a leaf)
        self.impurity = impurity            # Impurity of the node (e.g., Gini or entropy)

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2, criterion='gini'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion = criterion
        self.root = None # The root node of the decision tree
        print(f"DecisionTree initialized with max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, and criterion='{self.criterion}'")

    def _gini(self, y):
        m = len(y)
        if m == 0: # Handle empty set of labels
            return 0.0
        counts = Counter(y)
        gini = 1.0
        for label in counts:
            probability = counts[label] / m
            gini -= probability**2
        return gini

    def _entropy(self, y):
        m = len(y)
        if m == 0:
            return 0.0
        counts = Counter(y)
        entropy = 0.0
        for label in counts:
            probability = counts[label] / m
            entropy -= probability * np.log2(probability)
        return entropy

    def _find_best_split(self, X, y):
        n_samples, n_features = X.shape
        best_gini = float('inf')
        best_entropy = float('inf')
        best_split = None

        current_impurity = 0.0
        if self.criterion == 'gini':
            current_impurity = self._gini(y)
        else:
            current_impurity = self._entropy(y)

        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                # Split data based on the threshold
                left_indices = np.where(X[:, feature_index] <= threshold)[0]
                right_indices = np.where(X[:, feature_index] > threshold)[0]

                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue

                y_left, y_right = y[left_indices], y[right_indices]

                if self.criterion == 'gini':
                    gini_left = self._gini(y_left)
                    gini_right = self._gini(y_right)

                    # Weighted average of Gini impurities
                    weighted_gini = (len(y_left) / n_samples) * gini_left + \
                                    (len(y_right) / n_samples) * gini_right

                    if weighted_gini < best_gini:
                        best_gini = weighted_gini
                        best_split = {
                            'feature_index': feature_index,
                            'threshold': threshold,
                            'left_indices': left_indices,
                            'right_indices': right_indices
                        }
                elif self.criterion == 'entropy':
                    entropy_left = self._entropy(y_left)
                    entropy_right = self._entropy(y_right)

                    # Weighted average of entropy
                    weighted_entropy = (len(y_left) / n_samples) * entropy_left + \
                                       (len(y_right) / n_samples) * entropy_right

                    if weighted_entropy < best_entropy:
                        best_entropy = weighted_entropy
                        best_split = {
                            'feature_index': feature_index,
                            'threshold': threshold,
                            'left_indices': left_indices,
                            'right_indices': right_indices
                        }
        return best_split

**Reasoning**:
Implement a private helper method, `_build_tree`, that recursively constructs the decision tree. This method should handle stopping conditions such as `max_depth` reached, `min_samples_split` not met, or all samples belonging to the same class, as it's the core logic for tree construction.



In [ ]:
import numpy as np
from collections import Counter

class TreeNode:
    def __init__(self, feature_index=None, threshold=None, left_child=None, right_child=None, value=None, impurity=None):
        self.feature_index = feature_index  # Index of the feature used for splitting
        self.threshold = threshold          # Threshold value for splitting
        self.left_child = left_child        # Left child node
        self.right_child = right_child      # Right child node
        self.value = value                  # Class value for leaf node (if not None, it's a leaf)
        self.impurity = impurity            # Impurity of the node (e.g., Gini or entropy)

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2, criterion='gini'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion = criterion
        self.root = None # The root node of the decision tree
        # print(f"DecisionTree initialized with max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, and criterion='{self.criterion}'")

    def _gini(self, y):
        m = len(y)
        if m == 0: # Handle empty set of labels
            return 0.0
        counts = Counter(y)
        gini = 1.0
        for label in counts:
            probability = counts[label] / m
            gini -= probability**2
        return gini

    def _entropy(self, y):
        m = len(y)
        if m == 0:
            return 0.0
        counts = Counter(y)
        entropy = 0.0
        for label in counts:
            probability = counts[label] / m
            entropy -= probability * np.log2(probability)
        return entropy

    def _find_best_split(self, X, y):
        n_samples, n_features = X.shape
        if n_samples <= 1: # Cannot split if less than or equal to 1 sample
            return None

        best_gain = -1.0 # Maximize information gain
        best_split = None

        current_impurity = 0.0
        if self.criterion == 'gini':
            current_impurity = self._gini(y)
        else:
            current_impurity = self._entropy(y)

        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                left_indices = np.where(X[:, feature_index] <= threshold)[0]
                right_indices = np.where(X[:, feature_index] > threshold)[0]

                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue

                y_left, y_right = y[left_indices], y[right_indices]
                n_left, n_right = len(y_left), len(y_right)

                if self.criterion == 'gini':
                    impurity_left = self._gini(y_left)
                    impurity_right = self._gini(y_right)

                    weighted_impurity = (n_left / n_samples) * impurity_left + \
                                        (n_right / n_samples) * impurity_right
                    gain = current_impurity - weighted_impurity

                    if gain > best_gain:
                        best_gain = gain
                        best_split = {
                            'feature_index': feature_index,
                            'threshold': threshold,
                            'left_indices': left_indices,
                            'right_indices': right_indices
                        }
                elif self.criterion == 'entropy':
                    impurity_left = self._entropy(y_left)
                    impurity_right = self._entropy(y_right)

                    weighted_impurity = (n_left / n_samples) * impurity_left + \
                                       (n_right / n_samples) * impurity_right
                    gain = current_impurity - weighted_impurity

                    if gain > best_gain:
                        best_gain = gain
                        best_split = {
                            'feature_index': feature_index,
                            'threshold': threshold,
                            'left_indices': left_indices,
                            'right_indices': right_indices
                        }
        return best_split

    def _build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # Stopping conditions
        # 1. If all samples belong to the same class
        if n_labels == 1:
            return TreeNode(value=y[0], impurity=0.0)

        # 2. If max_depth is reached
        if self.max_depth is not None and depth >= self.max_depth:
            # Create a leaf node with the most common class
            leaf_value = Counter(y).most_common(1)[0][0]
            leaf_impurity = self._gini(y) if self.criterion == 'gini' else self._entropy(y)
            return TreeNode(value=leaf_value, impurity=leaf_impurity)

        # 3. If min_samples_split is not met
        if n_samples < self.min_samples_split:
            # Create a leaf node with the most common class
            leaf_value = Counter(y).most_common(1)[0][0]
            leaf_impurity = self._gini(y) if self.criterion == 'gini' else self._entropy(y)
            return TreeNode(value=leaf_value, impurity=leaf_impurity)

        # Find the best split
        best_split = self._find_best_split(X, y)

        # If no split improves impurity, make it a leaf node
        if best_split is None or best_split.get('gain', 0) <= 0: # Assuming gain was calculated and returned
            leaf_value = Counter(y).most_common(1)[0][0]
            leaf_impurity = self._gini(y) if self.criterion == 'gini' else self._entropy(y)
            return TreeNode(value=leaf_value, impurity=leaf_impurity)

        # Recursive calls to build left and right subtrees
        left_child = self._build_tree(X[best_split['left_indices']], y[best_split['left_indices']], depth + 1)
        right_child = self._build_tree(X[best_split['right_indices']], y[best_split['right_indices']], depth + 1)

        # Calculate impurity for the current node based on the split
        node_impurity = self._gini(y) if self.criterion == 'gini' else self._entropy(y)

        return TreeNode(
            feature_index=best_split['feature_index'],
            threshold=best_split['threshold'],
            left_child=left_child,
            right_child=right_child,
            impurity=node_impurity
        )


**Reasoning**:
Implement the `fit` method that takes `X` (features) and `y` (labels) as input and calls `_build_tree` to construct the decision tree, which is necessary to train the model.



In [ ]:
import numpy as np
from collections import Counter

class TreeNode:
    def __init__(self, feature_index=None, threshold=None, left_child=None, right_child=None, value=None, impurity=None):
        self.feature_index = feature_index  # Index of the feature used for splitting
        self.threshold = threshold          # Threshold value for splitting
        self.left_child = left_child        # Left child node
        self.right_child = right_child      # Right child node
        self.value = value                  # Class value for leaf node (if not None, it's a leaf)
        self.impurity = impurity            # Impurity of the node (e.g., Gini or entropy)

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2, criterion='gini'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion = criterion
        self.root = None # The root node of the decision tree
        # print(f"DecisionTree initialized with max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, and criterion='{self.criterion}'")

    def _gini(self, y):
        m = len(y)
        if m == 0: # Handle empty set of labels
            return 0.0
        counts = Counter(y)
        gini = 1.0
        for label in counts:
            probability = counts[label] / m
            gini -= probability**2
        return gini

    def _entropy(self, y):
        m = len(y)
        if m == 0:
            return 0.0
        counts = Counter(y)
        entropy = 0.0
        for label in counts:
            probability = counts[label] / m
            entropy -= probability * np.log2(probability)
        return entropy

    def _find_best_split(self, X, y):
        n_samples, n_features = X.shape
        if n_samples <= 1: # Cannot split if less than or equal to 1 sample
            return None

        best_gain = -1.0 # Maximize information gain
        best_split = None

        current_impurity = 0.0
        if self.criterion == 'gini':
            current_impurity = self._gini(y)
        else:
            current_impurity = self._entropy(y)

        for feature_index in range(n_features):
            # Use a limited number of unique values for thresholds to speed up computation
            # Or consider a subset of feature values as potential thresholds
            unique_values = np.unique(X[:, feature_index])
            # For large datasets, sampling thresholds can be an option
            # if len(unique_values) > self.max_thresholds_to_check:
            #    thresholds = np.random.choice(unique_values, self.max_thresholds_to_check, replace=False)
            # else:
            thresholds = unique_values

            for threshold in thresholds:
                left_indices = np.where(X[:, feature_index] <= threshold)[0]
                right_indices = np.where(X[:, feature_index] > threshold)[0]

                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue

                y_left, y_right = y[left_indices], y[right_indices]
                n_left, n_right = len(y_left), len(y_right)

                if self.criterion == 'gini':
                    impurity_left = self._gini(y_left)
                    impurity_right = self._gini(y_right)

                    weighted_impurity = (n_left / n_samples) * impurity_left + \
                                        (n_right / n_samples) * impurity_right
                    gain = current_impurity - weighted_impurity

                    if gain > best_gain:
                        best_gain = gain
                        best_split = {
                            'feature_index': feature_index,
                            'threshold': threshold,
                            'left_indices': left_indices,
                            'right_indices': right_indices,
                            'gain': gain
                        }
                elif self.criterion == 'entropy':
                    impurity_left = self._entropy(y_left)
                    impurity_right = self._entropy(y_right)

                    weighted_impurity = (n_left / n_samples) * impurity_left + \
                                       (n_right / n_samples) * impurity_right
                    gain = current_impurity - weighted_impurity

                    if gain > best_gain:
                        best_gain = gain
                        best_split = {
                            'feature_index': feature_index,
                            'threshold': threshold,
                            'left_indices': left_indices,
                            'right_indices': right_indices,
                            'gain': gain
                        }
        return best_split

    def _build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # Stopping conditions
        # 1. If all samples belong to the same class
        if n_labels == 1:
            return TreeNode(value=y[0], impurity=0.0)

        # 2. If max_depth is reached
        if self.max_depth is not None and depth >= self.max_depth:
            # Create a leaf node with the most common class
            leaf_value = Counter(y).most_common(1)[0][0]
            leaf_impurity = self._gini(y) if self.criterion == 'gini' else self._entropy(y)
            return TreeNode(value=leaf_value, impurity=leaf_impurity)

        # 3. If min_samples_split is not met
        if n_samples < self.min_samples_split:
            # Create a leaf node with the most common class
            leaf_value = Counter(y).most_common(1)[0][0]
            leaf_impurity = self._gini(y) if self.criterion == 'gini' else self._entropy(y)
            return TreeNode(value=leaf_value, impurity=leaf_impurity)

        # Find the best split
        best_split = self._find_best_split(X, y)

        # If no split improves impurity (or no valid split found), make it a leaf node
        if best_split is None or best_split.get('gain', 0) <= 0:
            leaf_value = Counter(y).most_common(1)[0][0]
            leaf_impurity = self._gini(y) if self.criterion == 'gini' else self._entropy(y)
            return TreeNode(value=leaf_value, impurity=leaf_impurity)

        # Recursive calls to build left and right subtrees
        left_child = self._build_tree(X[best_split['left_indices']], y[best_split['left_indices']], depth + 1)
        right_child = self._build_tree(X[best_split['right_indices']], y[best_split['right_indices']], depth + 1)

        # Calculate impurity for the current node based on the split
        node_impurity = self._gini(y) if self.criterion == 'gini' else self._entropy(y)

        return TreeNode(
            feature_index=best_split['feature_index'],
            threshold=best_split['threshold'],
            left_child=left_child,
            right_child=right_child,
            impurity=node_impurity
        )

    def fit(self, X, y):
        # Ensure X and y are numpy arrays for consistent indexing
        self.X_train = X.to_numpy() if hasattr(X, 'to_numpy') else X
        self.y_train = y.to_numpy() if hasattr(y, 'to_numpy') else y
        self.root = self._build_tree(self.X_train, self.y_train)
        print("DecisionTree fitted successfully.")

**Reasoning**:
Implement a private helper method, `_predict_one_sample`, within the `DecisionTree` class. This method will recursively traverse the trained tree from the root down to a leaf node, making decisions at each internal node based on the feature and threshold, and returning the predicted class value from the leaf node.



In [ ]:
import numpy as np
from collections import Counter

class TreeNode:
    def __init__(self, feature_index=None, threshold=None, left_child=None, right_child=None, value=None, impurity=None):
        self.feature_index = feature_index  # Index of the feature used for splitting
        self.threshold = threshold          # Threshold value for splitting
        self.left_child = left_child        # Left child node
        self.right_child = right_child      # Right child node
        self.value = value                  # Class value for leaf node (if not None, it's a leaf)
        self.impurity = impurity            # Impurity of the node (e.g., Gini or entropy)

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2, criterion='gini'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion = criterion
        self.root = None # The root node of the decision tree
        # print(f"DecisionTree initialized with max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, and criterion='{self.criterion}'")

    def _gini(self, y):
        m = len(y)
        if m == 0: # Handle empty set of labels
            return 0.0
        counts = Counter(y)
        gini = 1.0
        for label in counts:
            probability = counts[label] / m
            gini -= probability**2
        return gini

    def _entropy(self, y):
        m = len(y)
        if m == 0:
            return 0.0
        counts = Counter(y)
        entropy = 0.0
        for label in counts:
            probability = counts[label] / m
            entropy -= probability * np.log2(probability)
        return entropy

    def _find_best_split(self, X, y):
        n_samples, n_features = X.shape
        if n_samples <= 1: # Cannot split if less than or equal to 1 sample
            return None

        best_gain = -1.0 # Maximize information gain
        best_split = None

        current_impurity = 0.0
        if self.criterion == 'gini':
            current_impurity = self._gini(y)
        else:
            current_impurity = self._entropy(y)

        for feature_index in range(n_features):
            # Use a limited number of unique values for thresholds to speed up computation
            # Or consider a subset of feature values as potential thresholds
            unique_values = np.unique(X[:, feature_index])
            # For large datasets, sampling thresholds can be an option
            # if len(unique_values) > self.max_thresholds_to_check:
            #    thresholds = np.random.choice(unique_values, self.max_thresholds_to_check, replace=False)
            # else:
            thresholds = unique_values

            for threshold in thresholds:
                left_indices = np.where(X[:, feature_index] <= threshold)[0]
                right_indices = np.where(X[:, feature_index] > threshold)[0]

                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue

                y_left, y_right = y[left_indices], y[right_indices]
                n_left, n_right = len(y_left), len(y_right)

                if self.criterion == 'gini':
                    impurity_left = self._gini(y_left)
                    impurity_right = self._gini(y_right)

                    weighted_impurity = (n_left / n_samples) * impurity_left + \
                                        (n_right / n_samples) * impurity_right
                    gain = current_impurity - weighted_impurity

                    if gain > best_gain:
                        best_gain = gain
                        best_split = {
                            'feature_index': feature_index,
                            'threshold': threshold,
                            'left_indices': left_indices,
                            'right_indices': right_indices,
                            'gain': gain
                        }
                elif self.criterion == 'entropy':
                    impurity_left = self._entropy(y_left)
                    impurity_right = self._entropy(y_right)

                    weighted_impurity = (n_left / n_samples) * impurity_left + \
                                       (n_right / n_samples) * impurity_right
                    gain = current_impurity - weighted_impurity

                    if gain > best_gain:
                        best_gain = gain
                        best_split = {
                            'feature_index': feature_index,
                            'threshold': threshold,
                            'left_indices': left_indices,
                            'right_indices': right_indices,
                            'gain': gain
                        }
        return best_split

    def _build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # Stopping conditions
        # 1. If all samples belong to the same class
        if n_labels == 1:
            return TreeNode(value=y[0], impurity=0.0)

        # 2. If max_depth is reached
        if self.max_depth is not None and depth >= self.max_depth:
            # Create a leaf node with the most common class
            leaf_value = Counter(y).most_common(1)[0][0]
            leaf_impurity = self._gini(y) if self.criterion == 'gini' else self._entropy(y)
            return TreeNode(value=leaf_value, impurity=leaf_impurity)

        # 3. If min_samples_split is not met
        if n_samples < self.min_samples_split:
            # Create a leaf node with the most common class
            leaf_value = Counter(y).most_common(1)[0][0]
            leaf_impurity = self._gini(y) if self.criterion == 'gini' else self._entropy(y)
            return TreeNode(value=leaf_value, impurity=leaf_impurity)

        # Find the best split
        best_split = self._find_best_split(X, y)

        # If no split improves impurity (or no valid split found), make it a leaf node
        if best_split is None or best_split.get('gain', 0) <= 0:
            leaf_value = Counter(y).most_common(1)[0][0]
            leaf_impurity = self._gini(y) if self.criterion == 'gini' else self._entropy(y)
            return TreeNode(value=leaf_value, impurity=leaf_impurity)

        # Recursive calls to build left and right subtrees
        left_child = self._build_tree(X[best_split['left_indices']], y[best_split['left_indices']], depth + 1)
        right_child = self._build_tree(X[best_split['right_indices']], y[best_split['right_indices']], depth + 1)

        # Calculate impurity for the current node based on the split
        node_impurity = self._gini(y) if self.criterion == 'gini' else self._entropy(y)

        return TreeNode(
            feature_index=best_split['feature_index'],
            threshold=best_split['threshold'],
            left_child=left_child,
            right_child=right_child,
            impurity=node_impurity
        )

    def fit(self, X, y):
        # Ensure X and y are numpy arrays for consistent indexing
        self.X_train = X.to_numpy() if hasattr(X, 'to_numpy') else X
        self.y_train = y.to_numpy() if hasattr(y, 'to_numpy') else y
        self.root = self._build_tree(self.X_train, self.y_train)
        print("DecisionTree fitted successfully.")

    def _predict_one_sample(self, x, node):
        # If it's a leaf node, return its value
        if node.value is not None:
            return node.value

        # Traverse left or right based on the feature and threshold
        if x[node.feature_index] <= node.threshold:
            return self._predict_one_sample(x, node.left_child)
        else:
            return self._predict_one_sample(x, node.right_child)


**Reasoning**:
Implement the `predict` method within the `DecisionTree` class. This method will take an array of feature vectors `X_test` and return an array of predictions by iterating through each sample and calling the `_predict_one_sample` helper method for each.



In [ ]:
import numpy as np
from collections import Counter

class TreeNode:
    def __init__(self, feature_index=None, threshold=None, left_child=None, right_child=None, value=None, impurity=None):
        self.feature_index = feature_index  # Index of the feature used for splitting
        self.threshold = threshold          # Threshold value for splitting
        self.left_child = left_child        # Left child node
        self.right_child = right_child      # Right child node
        self.value = value                  # Class value for leaf node (if not None, it's a leaf)
        self.impurity = impurity            # Impurity of the node (e.g., Gini or entropy)

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2, criterion='gini'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion = criterion
        self.root = None # The root node of the decision tree
        # print(f"DecisionTree initialized with max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, and criterion='{self.criterion}'")

    def _gini(self, y):
        m = len(y)
        if m == 0: # Handle empty set of labels
            return 0.0
        counts = Counter(y)
        gini = 1.0
        for label in counts:
            probability = counts[label] / m
            gini -= probability**2
        return gini

    def _entropy(self, y):
        m = len(y)
        if m == 0:
            return 0.0
        counts = Counter(y)
        entropy = 0.0
        for label in counts:
            probability = counts[label] / m
            entropy -= probability * np.log2(probability)
        return entropy

    def _find_best_split(self, X, y):
        n_samples, n_features = X.shape
        if n_samples <= 1: # Cannot split if less than or equal to 1 sample
            return None

        best_gain = -1.0 # Maximize information gain
        best_split = None

        current_impurity = 0.0
        if self.criterion == 'gini':
            current_impurity = self._gini(y)
        else:
            current_impurity = self._entropy(y)

        for feature_index in range(n_features):
            # Use a limited number of unique values for thresholds to speed up computation
            # Or consider a subset of feature values as potential thresholds
            unique_values = np.unique(X[:, feature_index])
            # For large datasets, sampling thresholds can be an option
            # if len(unique_values) > self.max_thresholds_to_check:
            #    thresholds = np.random.choice(unique_values, self.max_thresholds_to_check, replace=False)
            # else:
            thresholds = unique_values

            for threshold in thresholds:
                left_indices = np.where(X[:, feature_index] <= threshold)[0]
                right_indices = np.where(X[:, feature_index] > threshold)[0]

                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue

                y_left, y_right = y[left_indices], y[right_indices]
                n_left, n_right = len(y_left), len(y_right)

                if self.criterion == 'gini':
                    impurity_left = self._gini(y_left)
                    impurity_right = self._gini(y_right)

                    weighted_impurity = (n_left / n_samples) * impurity_left + \
                                        (n_right / n_samples) * impurity_right
                    gain = current_impurity - weighted_impurity

                    if gain > best_gain:
                        best_gain = gain
                        best_split = {
                            'feature_index': feature_index,
                            'threshold': threshold,
                            'left_indices': left_indices,
                            'right_indices': right_indices,
                            'gain': gain
                        }
                elif self.criterion == 'entropy':
                    impurity_left = self._entropy(y_left)
                    impurity_right = self._entropy(y_right)

                    weighted_impurity = (n_left / n_samples) * impurity_left + \
                                       (n_right / n_samples) * impurity_right
                    gain = current_impurity - weighted_impurity

                    if gain > best_gain:
                        best_gain = gain
                        best_split = {
                            'feature_index': feature_index,
                            'threshold': threshold,
                            'left_indices': left_indices,
                            'right_indices': right_indices,
                            'gain': gain
                        }
        return best_split

    def _build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # Stopping conditions
        # 1. If all samples belong to the same class
        if n_labels == 1:
            return TreeNode(value=y[0], impurity=0.0)

        # 2. If max_depth is reached
        if self.max_depth is not None and depth >= self.max_depth:
            # Create a leaf node with the most common class
            leaf_value = Counter(y).most_common(1)[0][0]
            leaf_impurity = self._gini(y) if self.criterion == 'gini' else self._entropy(y)
            return TreeNode(value=leaf_value, impurity=leaf_impurity)

        # 3. If min_samples_split is not met
        if n_samples < self.min_samples_split:
            # Create a leaf node with the most common class
            leaf_value = Counter(y).most_common(1)[0][0]
            leaf_impurity = self._gini(y) if self.criterion == 'gini' else self._entropy(y)
            return TreeNode(value=leaf_value, impurity=leaf_impurity)

        # Find the best split
        best_split = self._find_best_split(X, y)

        # If no split improves impurity (or no valid split found), make it a leaf node
        if best_split is None or best_split.get('gain', 0) <= 0:
            leaf_value = Counter(y).most_common(1)[0][0]
            leaf_impurity = self._gini(y) if self.criterion == 'gini' else self._entropy(y)
            return TreeNode(value=leaf_value, impurity=leaf_impurity)

        # Recursive calls to build left and right subtrees
        left_child = self._build_tree(X[best_split['left_indices']], y[best_split['left_indices']], depth + 1)
        right_child = self._build_tree(X[best_split['right_indices']], y[best_split['right_indices']], depth + 1)

        # Calculate impurity for the current node based on the split
        node_impurity = self._gini(y) if self.criterion == 'gini' else self._entropy(y)

        return TreeNode(
            feature_index=best_split['feature_index'],
            threshold=best_split['threshold'],
            left_child=left_child,
            right_child=right_child,
            impurity=node_impurity
        )

    def fit(self, X, y):
        # Ensure X and y are numpy arrays for consistent indexing
        self.X_train = X.to_numpy() if hasattr(X, 'to_numpy') else X
        self.y_train = y.to_numpy() if hasattr(y, 'to_numpy') else y
        self.root = self._build_tree(self.X_train, self.y_train)
        print("DecisionTree fitted successfully.")

    def _predict_one_sample(self, x, node):
        # If it's a leaf node, return its value
        if node.value is not None:
            return node.value

        # Traverse left or right based on the feature and threshold
        if x[node.feature_index] <= node.threshold:
            return self._predict_one_sample(x, node.left_child)
        else:
            return self._predict_one_sample(x, node.right_child)

    def predict(self, X):
        # Ensure X is a numpy array for consistent indexing
        X_np = X.to_numpy() if hasattr(X, 'to_numpy') else X
        return np.array([self._predict_one_sample(x, self.root) for x in X_np])


## Implement Random Forest Classifier

### Subtask:
Define the RandomForestClassifier class with an __init__ method to initialize parameters such as the number of estimators, maximum depth for individual trees, minimum samples required to split a node, impurity criterion, and a random state for reproducibility.


**Reasoning**:
Implement the `RandomForestClassifier` class with an `__init__` method to initialize parameters required for building a Random Forest, such as the number of estimators, max depth, min samples split, criterion, and a random state for reproducibility.



In [ ]:
import numpy as np
from collections import Counter

# Assuming TreeNode and DecisionTree classes are defined in previous cells
# (or will be defined as part of the overall RandomForest implementation)

class RandomForestClassifier:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2, criterion='gini', random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion = criterion
        self.random_state = random_state
        self.trees = [] # List to store individual DecisionTree objects
        print(f"RandomForestClassifier initialized with n_estimators={self.n_estimators}, max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, criterion='{self.criterion}', and random_state={self.random_state}")